In [39]:
from socceraction.xthreat import ExpectedThreat
import pandas as pd
import psycopg2


conn = psycopg2.connect(
    host="fuji.ucll.be",
    database="international_week",
    user="busit_79",
    password="[KSiO^`9??V\K=?W2>Z:`",
    port=52425,
    sslmode="require",
)

query = """
SELECT 
    s.id, s.game_id, s.period_id, s.seconds, 
    s.player_id, s.team_id, 
    s.start_x, s.start_y, s.end_x, s.end_y, 
    s.action_type, s.result, s.bodypart,
    p.player_name, t.team_name
FROM 
    spadl_actions s
LEFT JOIN 
    players p ON s.player_id = p.player_id
LEFT JOIN 
    teams t ON s.team_id = t.team_id
WHERE 
    s.start_x IS NOT NULL 
    AND s.end_x IS NOT NULL
    AND s.action_type IN ('1', '2', '3', '4', '5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21')
"""

# Load data into pandas DataFrame
df = pd.read_sql(query, conn)


# Preview the dataset
print(df.head())

C:\Users\purva\AppData\Local\Temp\ipykernel_39620\2552419504.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   id                    game_id  period_id  seconds  \
0   2  5oc8drrbruovbuiriyhdyiyok          1      1.0   
1   5  5oc8drrbruovbuiriyhdyiyok          1     19.0   
2   7  5oc8drrbruovbuiriyhdyiyok          1     21.0   
3   9  5oc8drrbruovbuiriyhdyiyok          1     23.0   
4  10  5oc8drrbruovbuiriyhdyiyok          1     25.0   

                   player_id                    team_id  start_x  start_y  \
0   l6y4o9t2b0jxaxo85oztok45  1oyb7oym5nwzny8vxf03szd2h   32.865   30.464   
1   4kuxth5k98x4cgfrfweu1u51  b7jmo07lqav0wfe2mtlzyspak   62.265   68.000   
2  bybggxgpzlr7xz2f1i3b7c0kp  1oyb7oym5nwzny8vxf03szd2h   36.225   46.988   
3  8mkkchf0ysdf78r0lncd73klw  1oyb7oym5nwzny8vxf03szd2h   47.880   27.812   
4  8mkkchf0ysdf78r0lncd73klw  1oyb7oym5nwzny8vxf03szd2h   47.040   31.620   

    end_x   end_y action_type result bodypart  player_name    team_name  
0  32.445  33.864          21      1        0   B. Mechele  Club Brugge  
1  37.380  56.644           2      0        2     S.

In [49]:
# Initialize Expected Threat model
xT_model = ExpectedThreat()

# Convert DataFrame columns to the format expected by the model
# socceraction expects normalized coordinates (0-1)
pitch_length = 105  # Standard soccer pitch length in meters
pitch_width = 68    # Standard soccer pitch width in meters

# Normalize coordinates if they're not already normalized
if df['start_x'].max() > 1 or df['end_x'].max() > 1:
    df['start_x_norm'] = df['start_x'] / pitch_length
    df['start_y_norm'] = df['start_y'] / pitch_width
    df['end_x_norm'] = df['end_x'] / pitch_length  
    df['end_y_norm'] = df['end_y'] / pitch_width
else:
    df['start_x_norm'] = df['start_x']
    df['start_y_norm'] = df['start_y']
    df['end_x_norm'] = df['end_x']
    df['end_y_norm'] = df['end_y']

# Calculate xT values directly using the model (this avoids the need to manually index)
df['start_xT'] = xT_model.xT[df['start_x_norm'].round(2)*xT_model.l, df['start_y_norm'].round(2)*xT_model.w]
df['end_xT'] = xT_model.xT[df['end_x_norm'].round(2)*xT_model.l, df['end_y_norm'].round(2)*xT_model.w]

# Compute xT contribution
df['xT_contribution'] = df['end_xT'] - df['start_xT']

# Show top 10 xT contributions with player and team names
print(df[['player_name', 'team_name', 'xT_contribution']].sort_values(by='xT_contribution', ascending=False).head(10))


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices